In [1]:
import sys,tweepy,csv,re
import matplotlib.pyplot as plt
import string
import nltk
import Sastrawi
import numpy as np

In [2]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [4]:
class SentimentAnalysis:
    
    def __init__(self):
        self.tweets = []
        self.tweetText = []
    def DownloadData(self,searchTerm, NoOfTerms):
        # authenticating
        print(searchTerm, NoOfTerms)
        consumerKey='qubRP6d5B5eCD3RHrgoMRkDjH'
        consumerSecret='AJlk1MiHfXqnhByWtCf1Mbi2T9WGW08hoLkkDIbks4ztUpFx99'
        accessToken='3180202308-DmrgiSoxUUoSRevLLPldgCE57M0eD117K0mhFG5'
        accessTokenSecret='fuY5Hkk838OnmGOZUb16KLkSDw3JZjflxXQ1NHP6pP9Jr'
        auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
        auth.set_access_token(accessToken, accessTokenSecret)
        api = tweepy.API(auth, wait_on_rate_limit = False)

        # input for term to be searched and how many tweets to search
        # searchTerm = input("Enter Keyword/Tag to search about: ")
        # NoOfTerms = int(input("Enter how many tweets to search: "))

        # searching for tweets
        self.tweets = tweepy.Cursor(api.search_tweets, q=searchTerm, lang = "in").items(NoOfTerms)

        # Open/create a file to append data to
        csvFile = open('result.csv', 'a')

        # Use csv writer
        csvWriter = csv.writer(csvFile)

In [5]:
        # creating some variables to store info
        polarity = 0
        positive = 0
        negative = 0
        neutral = 0

In [6]:
        # iterating through tweets fetched
        for tweet in self.tweets:
            #Append to temp so that we can store in csv later. I use encode UTF-8
            self.tweetText.append(self.SVMmethod(tweet.text).encode('utf-8'))
            # print (tweet.text.translate(non_bmp_map))    #print tweet's text
            analysis = TextBlob(tweet.text)

            # print(analysis.sentiment)  # print tweet's polarity
            polarity += analysis.sentiment.polarity  # adding up polarities to find the average later
            if (analysis.sentiment.polarity == 0):  # adding reaction of how people are reacting to find average later
                neutral += 1
            
            elif (analysis.sentiment.polarity > 0 ):
                positive += 1
            
            elif (analysis.sentiment.polarity < 0):
                negative += 1
            

        # Write to csv and close csv file
        csvWriter.writerow(self.tweetText)
        csvFile.close()

        # finding average of how people are reacting
        positive = self.percentage(positive, NoOfTerms)
       
        negative = self.percentage(negative, NoOfTerms)
        
        neutral = self.percentage(neutral, NoOfTerms)

        # finding average reaction
        polarity = polarity / NoOfTerms

        # printing out data
        print("How people are reacting on " + searchTerm + " by analyzing " + str(NoOfTerms) + " tweets.")
        print()
        print("General Report: ")
        if (polarity == 0):
            print("Neutral")
        
        elif (polarity > 0 ):
            print("Positive")
       
        elif (polarity < 0):
            print("Negative")
        
        print()
        print("Detailed Report: ")
        print(str(positive) + "% people thought it was positive")
        
        print(str(negative) + "% people thought it was negative")
        
        print(str(neutral) + "% people thought it was neutral")
        self.plotPieChart(positive, negative, neutral, searchTerm, NoOfTerms)
        print('Accuracy score: ', accuracy_score(y_valid, y_pred))
        self.SVMmethod(positive, negative, neutral)
    def load_stopwords(self):
        list_stopwords = stopwords.words('indonesian')
        list_stopwords.extend(['yg', 'dg', 'dgn', 'ny', 'd', 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh',
                       '&amp', 'yah', 'klau', 'a', 'b', 'c', 'd', 'e', 'f',
                       'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 
                       's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
        list_stopwords = set(list_stopwords)
        return list_stopwords
    def cleanTweet(self, tweet):
        tweet = re.sub(r"\b[a-zA-Z]\b", "", tweet)
        tweet = re.sub('[0-9]+', '', tweet)
        tweet = tweet.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
        tweet = tweet.encode('ascii', 'replace').decode('ascii')
        tweet = re.sub(r'#', '', tweet)
        tweet = tweet.translate(str.maketrans("","",string.punctuation))
        tweet = tweet.strip()
        tweet = re.sub('\s+',' ',tweet)
        tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w +:\ / \ / \S +)", " ", tweet).split())
        tweet = word_tokenize(tweet)
        if tweet not in stopwords.words('indonesian'):
            return tweet
        else:
            return ''
        tweet = [stemmer.stem(w) for w in tweet.split()]
        return tweet
    sencol =[]
    senrow =np.array([])
    nsen = 0
    sentiment_list = []
    def Lexiconmethod(self, tweet):
        for i in range(len(df)):
            nsen = senrow.shape[0]
            words = word_tokenize(tweet)
            sentiment = 0 
            add = 0
            prev = [0 for ii in range(len(words))]
            n_words = len(words)
            if len(sencol)>0:
                sen =[0 for j in range(len(sencol))]
            else:
                sen =[]
                for word in words:
                    ind = words.index(word)
                    if word in lexicon_word :
                        sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
                    else:
                        kata_dasar = stemmer.stem(word)
                        if kata_dasar in lexicon_word:
                            sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
                        elif(n_words>1):
                            if ind-1>-1:
                                back_1    = words[ind-1]+' '+word
                                if (back_1 in lexicon_word):
                                    sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                                elif(ind-2>-1):
                                    back_2    = words[ind-2]+' '+back_1
                                    if back_2 in lexicon_word:
                                        sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
                                        if add>0:  
                                            if i>0:
                                                if (nsen==0):
                                                    senrow = np.zeros([i,add],dtype=int)
                                                elif(i!=nsen):
                                                    padding_h = np.zeros([nsen,add],dtype=int)
                                                    senrow = np.hstack((senrow,padding_h))
                                                    padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                                                    senrow = np.vstack((senrow,padding_v))
                                                else:
                                                    padding =np.zeros([nsen,add],dtype=int)
                                                    senrow = np.hstack((senrow,padding))
                                                    senrow = np.vstack((senrow,sen))
                                                    if i==0:
                                                        senrow = np.array(sen).reshape(1,len(sen))
                                                    elif(nsen>0):
                                                        senrow = np.vstack((senrow,sen))
                                                        sentiment_list.append(sentiment)
    def SVMmethod(self,tweet):
        train, valid = train_test_split(tweet, test_size=0.1, random_state=0)
        tfidf_vectorizer=TfidfVectorizer()
        tfidf_train=tfidf_vectorizer.fit_transform(X_train) 
        tfidf_test=tfidf_vectorizer.transform(X_test)
        clf = svm.SVC(kernel='linear', probability=True)
        clf.fit(tfidf_train, y_train)
        y_pred = clf.fit(tfidf_train, y_train).predict(tfidf_test)
        cm = metrics.confusion_matrix(y_valid, y_pred)

    # function to calculate percentage
    def percentage(self, part, whole):
        temp = 100 * float(part) / float(whole)
        return format(temp, '.2f')
    def plotPieChart(self, positive, negative, neutral, searchTerm, noOfSearchTerms):
        labels = ['Positive [' + str(positive) + '%]', 'Neutral [' + str(neutral) + '%]',
                  'Negative [' + str(negative) + '%]']
        sizes = [positive, neutral, negative]
        colors = ['brown', 'lightcoral', 'beige']
        patches, texts = plt.pie(sizes, colors=colors, startangle=90)
        plt.legend(patches, labels, loc="best")
        plt.title('How people are reacting on "' + searchTerm + '" by analyzing ' + str(noOfSearchTerms) + ' Tweets.')
        plt.axis('equal')
        plt.tight_layout()
        plt.savefig("static/fig.png")

IndentationError: unexpected indent (Temp/ipykernel_7480/3533646795.py, line 57)

if __name__== "__main__":<br>
    sa = SentimentAnalysis()<br>
    sa.DownloadData()